In [7]:
import numpy as np
import pandas as pd
import os 
import tensorflow as tf
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
import numpy as np
import plotly.offline as of
import plotly.graph_objs as go
of.offline.init_notebook_mode(connected=True)
import h5py
from data.plot3D import plot_voxelgrid, plot_points
from  data.voxelgrid import VoxelGrid

In [8]:
# with h5py.File("data/train_point_clouds.h5", "r") as points_dataset:        
#     digits = []
#     for i in range(10):
#         digit = (points_dataset[str(i)]["img"][:], 
#                  points_dataset[str(i)]["points"][:], 
#                  points_dataset[str(i)].attrs["label"]) 
#         digits.append(digit)
        
# x_c = [r[0] for r in digits[0][1]]
# y_c = [r[1] for r in digits[0][1]]
# z_c = [r[2] for r in digits[0][1]]
# trace1 = go.Scatter3d(x=x_c, y=y_c, z=z_c, mode='markers', 
#                       marker=dict(size=12, color=z_c, colorscale='Viridis', opacity=0.7))

# data = [trace1]
# layout = go.Layout(height=500, width=600, title= "Digit: "+str(digits[0][2]) + " in 3D space")
# fig = go.Figure(data=data, layout=layout)
# of.plot(fig)

'temp-plot.html'

In [9]:
for dirname,_, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

with h5py.File('data/full_dataset_vectors.h5', 'r') as dataset:
    x_train = dataset["X_train"][:]
    x_test = dataset["X_test"][:]
    y_train = dataset["y_train"][:]
    y_test = dataset["y_test"][:]

print ("x_train shape: ", x_train.shape)
print ("y_train shape: ", y_train.shape)
print ("x_test shape:  ", x_test.shape)
print ("y_test shape:  ", y_test.shape)

xtrain = np.ndarray((x_train.shape[0], 4096, 3))
xtest = np.ndarray((x_test.shape[0], 4096, 3))
def add_rgb_dimention(array):
    scaler_map = cm.ScalarMappable(cmap="Oranges")
    array = scaler_map.to_rgba(array)[:, : -1]
    return array

for i in range(x_train.shape[0]):
    xtrain[i] = add_rgb_dimention(x_train[i])
for i in range(x_test.shape[0]):
    xtest[i] = add_rgb_dimention(x_test[i])


xtrain = xtrain.reshape(x_train.shape[0], 16, 16, 16, 3)
xtest = xtest.reshape(x_test.shape[0], 16, 16, 16, 3)

print(xtrain.shape)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

data/plot3D.py
data/test_point_clouds.h5
data/full_dataset_vectors.h5
data/voxelgrid.py
data/train_point_clouds.h5
data/__pycache__/plot3D.cpython-37.pyc
data/__pycache__/voxelgrid.cpython-37.pyc
x_train shape:  (10000, 4096)
y_train shape:  (10000,)
x_test shape:   (2000, 4096)
y_test shape:   (2000,)
(10000, 16, 16, 16, 3)


In [58]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv3D(filters=8, kernel_size=(3, 3, 3),strides=1,padding ='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization(momentum=0.15, epsilon=1e-5, gamma_initializer="uniform"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv3D(filters=8, kernel_size=(3, 3, 3),strides=1,padding ='same',activation='relu'))
model.add(tf.keras.layers.BatchNormalization(momentum=0.15, epsilon=1e-5, gamma_initializer="uniform"))
model.add(tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=8192, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [59]:

adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=True)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
model.build(input_shape=(None, 16,16,16,3))
model.summary()


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_64 (Conv3D)           (None, 16, 16, 16, 8)     656       
_________________________________________________________________
batch_normalization_61 (Batc (None, 16, 16, 16, 8)     32        
_________________________________________________________________
dropout_117 (Dropout)        (None, 16, 16, 16, 8)     0         
_________________________________________________________________
conv3d_65 (Conv3D)           (None, 16, 16, 16, 8)     1736      
_________________________________________________________________
batch_normalization_62 (Batc (None, 16, 16, 16, 8)     32        
_________________________________________________________________
max_pooling3d_18 (MaxPooling (None, 8, 8, 8, 8)        0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 4096)            

In [60]:
model.fit(x=xtrain, y=y_train, batch_size=128, epochs=50, validation_split=0.2)

Epoch 1/50
63/63 [==============================] - 3s 45ms/step - loss: 1.5457 - acc: 0.4666 - val_loss: 1.2405 - val_acc: 0.5685
Epoch 2/50
63/63 [==============================] - 3s 45ms/step - loss: 1.1036 - acc: 0.6236 - val_loss: 1.1625 - val_acc: 0.6070
Epoch 3/50
63/63 [==============================] - 3s 42ms/step - loss: 0.9695 - acc: 0.6644 - val_loss: 1.1540 - val_acc: 0.6100
Epoch 4/50
63/63 [==============================] - 2s 39ms/step - loss: 0.8366 - acc: 0.7099 - val_loss: 1.0577 - val_acc: 0.6350
Epoch 5/50
63/63 [==============================] - 2s 39ms/step - loss: 0.7205 - acc: 0.7566 - val_loss: 1.0198 - val_acc: 0.6570
Epoch 6/50
63/63 [==============================] - 2s 39ms/step - loss: 0.5881 - acc: 0.8004 - val_loss: 1.0535 - val_acc: 0.6685
Epoch 7/50
63/63 [==============================] - 3s 40ms/step - loss: 0.4632 - acc: 0.8426 - val_loss: 1.2102 - val_acc: 0.6690
Epoch 8/50
63/63 [==============================] - 3s 40ms/step - loss: 0.3554 - a

In [61]:
model.evaluate(x=xtest,y=y_test)

63/63 [==============================] - 1s 9ms/step - loss: 2.0277 - acc: 0.6815


[2.0276589393615723, 0.6815000176429749]